This notebook generates a list of all the unique incidents tracked in the SIT 209 report
unique_fires.csv contains:
1. Incident Number	
2. Fire Name	
3. First Day
4. Last Day	
5. Lat Fire	
6. Lon Fire

In [2]:
import pandas as pd
#pd.set_option('display.max_rows', None)
import geopandas as gpd
import matplotlib.pyplot as plt
import os
import cartopy.crs as ccrs
import cartopy
import numpy as np
np.set_printoptions(threshold=33500)
from shapely.geometry import Polygon, Point, MultiPoint
from shapely.ops import cascaded_union
from datetime import datetime, timedelta

In [2]:
#load in all the data
sit_all =pd.read_csv('merged_sit.csv', dtype={'INCIDENT_JURISDICTION': 'str', 'UNIT_OR_OTHER_NARR': 'str'})
viirs_all = pd.read_csv('merged_viirs.csv', dtype={'satellite':'str', 'version': 'str'})

/home/lthapa/anaconda3/envs/ML_py/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3166: DtypeWarning: Columns (20,61) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
#generate the wildfire and complex codes
wildfire_codes = []
complex_codes = []
name_lookup ='SIT209_HISTORY_SIT209_LOOKUP_CODES_*.xlsx'
for year in range(2019, 2021):
    df_lookup = pd.read_excel(name_lookup.replace('*', str(year)))
    print(df_lookup.loc[df_lookup['CODE_NAME'] == 'Wildfire']['LUCODES_IDENTIFIER'].values)
    wildfire_codes.append(df_lookup.loc[df_lookup['CODE_NAME'] == 'Wildfire']['LUCODES_IDENTIFIER'].values[0])
    complex_codes.append(df_lookup.loc[df_lookup['CODE_NAME'] == 'Complex']['LUCODES_IDENTIFIER'].values[0])
print(wildfire_codes, complex_codes)

#select the wildfires and complexes only
sit_fire = sit_all[(sit_all['INCTYP_IDENTIFIER']==wildfire_codes[0])|(sit_all['INCTYP_IDENTIFIER']==wildfire_codes[1])|(sit_all['INCTYP_IDENTIFIER']==complex_codes[0])|(sit_all['INCTYP_IDENTIFIER']==complex_codes[1])]
print('We have ' + str(len(sit_fire['INCTYP_IDENTIFIER'])) + ' wildfire and complex reports')
#sanity check to see if we filtered out the right thing
aux = sit_all[(sit_all['INCTYP_IDENTIFIER']!=wildfire_codes[0])&(sit_all['INCTYP_IDENTIFIER']!=wildfire_codes[1])&(sit_all['INCTYP_IDENTIFIER']!=complex_codes[0])&(sit_all['INCTYP_IDENTIFIER']!=complex_codes[1])]
print(str(len(aux['INCTYP_IDENTIFIER']))+ ' Incidents were removed')

# find the unique incidents
inc_nums = sit_fire['INC_IDENTIFIER'].values # a numpy array of all the incidents
inc_names = sit_fire['INCIDENT_NAME'].values
inc_start_dates = sit_fire['DISCOVERY_DATE'].values

#compare number of unique incident numbers, names, and start dates
print('We have ' + str(len(np.unique(inc_nums))) + ' unique incident numbers, ' + str(len(np.unique(inc_names)))+ ' unique incident names, and ' + str(len(np.unique(inc_start_dates)))+ ' unique start dates')


[9352235]
[9429964]
[9352235, 9429964] [9352456, 9430185]
We have 17782 wildfire and complex reports
87 Incidents were removed
We have 2597 unique incident numbers, 2532 unique incident names, and 2920 unique start dates


In [4]:
# generate a dataframe of incidents that contains:
# the name, number, earliest discovery date as a day, end date of final report as a day

unique_incs = np.unique(inc_nums)
n_incs = len(unique_incs) #number of unique incidents
print(n_incs)
incidents = pd.DataFrame({'Incident Number': np.nan*np.zeros(n_incs), 
                          'Fire Name': np.nan*np.zeros(n_incs), 
                         'First Day': np.nan*np.zeros(n_incs),
                        'Last Day': np.nan*np.zeros(n_incs),
                        'Lat Fire': np.nan*np.zeros(n_incs), 
                     'Lon Fire': np.nan*np.zeros(n_incs)}) # pre-allocate the dataframe
for ii in range(n_incs):
    #grab the name and incident number
    thisinc_num = unique_incs[ii] # Incident Number
    df_fire = sit_fire[sit_fire['INC_IDENTIFIER']==thisinc_num]
    df_fire = df_fire.sort_values('REPORT_FROM_DATE')
    thisinc_startdate = np.unique(df_fire['DISCOVERY_DATE'].values)
    thisinc_firstday = np.amin(thisinc_startdate)[0:10] # First Day
    thisinc_enddate = df_fire['REPORT_TO_DATE'].values
    thisinc_lat = df_fire['POO_LATITUDE'].values[0]
    thisinc_lon = df_fire['POO_LONGITUDE'].values[0]
    if len(thisinc_enddate)>0:
        thisinc_lastday = np.amax(thisinc_enddate)[0:10] #get the last available day
    else:
        thisinc_lastday = np.nan
    
    #thisinc_lastday = thisinc_enddate[0:10] # Last Day
    thisinc_name = np.unique(df_fire['INCIDENT_NAME'].values) #Fire Name
    
    #fill in the whole row at once (format below)
    incidents.iloc[ii]= [thisinc_num, thisinc_name[0], thisinc_firstday, thisinc_lastday, thisinc_lat, thisinc_lon]
    
print(incidents)

2597
      Incident Number    Fire Name   First Day    Last Day   Lat Fire  \
0          10662063.0      Coleman  2019-05-04  2019-05-04  45.349722   
1          10662156.0          C34  2019-02-13  2019-02-15  36.995278   
2          10662179.0         2161  2019-02-13  2019-02-14  35.221389   
3          10662352.0    South 480  2019-01-31  2019-02-01  29.600278   
4          10662407.0      McClurg  2019-04-16  2019-04-22  36.781389   
...               ...          ...         ...         ...        ...   
2592       11980949.0      CREEK 5  2020-12-24  2020-12-24  33.407667   
2593       11980985.0     BIG SNAG  2020-12-26  2020-12-27  35.342778   
2594       11981035.0       Dubois  2020-12-27  2020-12-28  33.440000   
2595       11981111.0   FOOT PRINT  2020-12-28  2020-12-31  35.388333   
2596       11981205.0  SMALL PINES  2020-12-23  2020-12-24  29.388056   

        Lon Fire  
0     -87.886944  
1    -102.524722  
2    -101.518611  
3     -94.547222  
4     -92.750833  
... 

In [5]:
#check for nan values of the fire location, and drop them!
te = incidents['Lat Fire'].values
drops = np.where(np.isnan(te))[0]
incidents = incidents.drop(labels=drops)

In [6]:
# filter out fires whose starting location is outside of our domain
incidents = incidents[(incidents['Lat Fire']>=31)&(incidents['Lat Fire']<=49)&(incidents['Lon Fire']>=-125)&(incidents['Lon Fire']<=-101)]
print(incidents)

      Incident Number      Fire Name   First Day    Last Day   Lat Fire  \
1          10662156.0            C34  2019-02-13  2019-02-15  36.995278   
2          10662179.0           2161  2019-02-13  2019-02-14  35.221389   
11         10662684.0  PAINTED WAGON  2019-04-08  2019-04-10  33.786944   
20         10663171.0  LONE MOUNTAIN  2019-05-26  2019-07-03  33.808056   
21         10663525.0        BOULDER  2019-06-05  2019-06-06  35.333056   
...               ...            ...         ...         ...        ...   
2570       11979081.0        LAURA 2  2020-11-17  2020-11-23  40.034034   
2571       11979237.0          Trail  2020-10-24  2020-11-02  34.147222   
2583       11980186.0      SANDERSON  2020-12-13  2020-12-14  33.889444   
2588       11980587.0        CREEK 5  2020-12-24  2020-12-28  33.402646   
2592       11980949.0        CREEK 5  2020-12-24  2020-12-24  33.407667   

        Lon Fire  
1    -102.524722  
2    -101.518611  
11   -112.753333  
20   -105.738611  
21  

In [7]:
# save incidents as a csv file
incidents.to_csv('unique_fires.csv', index=False)